# Installing Libraries

In [ ]:
!pip install -q scikit-learn numpy pandas altair

# Cross-Validation
대부분의 Cross-Validation 방법은 [sklearn.model_selection](https://scikit-learn.org/stable/modules/classes.html#module-sklearn.model_selection)에 구현되어 있다.

아래의 함수는 Split이 어떻게 나뉘어져 있는지를 시각화하고 데이터 분포를 표시한다.

In [ ]:
import altair as alt
import numpy as np
import pandas as pd
from sklearn.model_selection import BaseCrossValidator


def vis_and_stats(splitter: BaseCrossValidator, X: np.ndarray, y: np.ndarray, group: np.ndarray = None, tick_options: dict = None ):
    indices, stats = [], []
    for i, (I_train, I_test) in enumerate(splitter.split(X, y, group)):
        y_train, y_test = y[I_train], y[I_test]
        name = f'{i + 1}-th split'
        indices_train = [(idx, 'Training set', name) for idx in I_train]
        indices_test = [(idx, 'Testing set', name) for idx in I_test]
        cls_train, cnt_train = np.unique(y_train, return_counts=True)
        cls_test, cnt_test = np.unique(y_test, return_counts=True)
        stat_train = {f'N.train.{k}': v for k, v in zip(cls_train, cnt_train)}
        stat_test = {f'N.test.{k}': v for k, v in zip(cls_test, cnt_test)}


        stat = {
            'split': name,
            'N.train': len(y_train),
            'N.test': len(y_test),
            **stat_train,
            **stat_test
        }

        indices.extend(indices_train)
        indices.extend(indices_test)
        stats.append(stat)
    indices = pd.DataFrame(indices, columns=['Index', 'Dataset', 'Split'])
    stats = pd.DataFrame(stats).assign(
        ratio_train=lambda x: x['N.train.0']/x['N.train.1'],
        ratio_test=lambda x: x['N.test.0']/x['N.test.1'],
    )
    alt.data_transformers.disable_max_rows()

    tick_options = tick_options or dict()
    tick_options = {
        'thickness': 1,
        'bandSize': 20,
        **tick_options
    }

    chart = alt.Chart(indices).mark_tick(
        **tick_options
    ).encode(
        x='Index:Q', y='Split:O', color=alt.Color('Dataset:N')
    ).properties(
        height=250
    )

    if group is not None:
        df_group = pd.DataFrame({
            (k, v, 'Group') for k, v in enumerate(group)
        }, columns=['Index', 'Group', 'Split'])

        chart_group = alt.Chart(df_group).mark_tick(
            **tick_options
        ).encode(
            x=alt.X('Index:Q', title=None),
            y=alt.Y('Split:O', title=None),
            color=alt.Color('Group:N')
        )
        chart = chart & chart_group

        chart = chart.resolve_scale(
            color='independent'
        )
    return chart, stats

## Hold-Out CV
가장 간단하게는, 전체 샘플 중 일부를 무작위로 추출하여 검증 데이터로 활용하는 Hold-Out CV를 해보자. 이미 실습에서도 많이 사용했듯이, [sklearn.model_selection.ShuffleSplit](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.ShuffleSplit.html#sklearn.model_selection.ShuffleSplit) 또는  [sklearn.model_selection.StratifiedShuffleSplit](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.StratifiedShuffleSplit.html#sklearn.model_selection.StratifiedShuffleSplit)을 활용하면 된다.

보통, Cross-Validation의 흐름은 아래와 같다.

In [ ]:
import numpy as np
from sklearn.datasets import make_classification
from sklearn.model_selection import ShuffleSplit
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import log_loss


X, y = make_classification(
    n_samples=1000, # 총 샘플 개수
    n_features=15, # 특성값 수
    n_redundant=1, # 다중공선성을 가지는 특성값 수
    n_clusters_per_class=1, # 레이블 값 별 데이터 군집의 수
    weights=[0.49, 0.51], # 레이블 값 비율
    flip_y=0, # 레이블 값에 적용할 노이즈 비율
    random_state=42
)

splitter = ShuffleSplit(
    n_splits=5, # 생성할 Split의 갯수
    test_size=0.4, # 전체 샘플의 40%를 검증 데이터로 활용
    random_state=42
)

scores_train, scores_test = [], []

# split 호출 시 훈련 데이터셋과 검증 데이터셋이 될 샘플의 인덱스들을 tuple 형식으로 반환하는 Generator가 생성된다.
for I_train, I_test in splitter.split(X, y):
    # 훈련 및 검증 데이터셋은 다음과 같은 방식으로 나누면 된다.
    X_train, X_test, y_train, y_test = X[I_train, :], X[I_test], y[I_train], y[I_test]

    # 훈련 데이터셋으로 모델을 훈련시킨다.
    model = LogisticRegression().fit(X=X_train, y=y_train)

    # 훈련이 잘 되었는지 확인해본다.
    score_train = log_loss(y_true=y_train, y_pred=model.predict_proba(X_train)[:, 1])

    # 검증 데이터셋을 통해 일반화 성능을 추정한다.
    score_test = log_loss(y_true=y_test, y_pred=model.predict_proba(X_test)[:, 1])

    scores_train.append(score_train)
    scores_test.append(score_test)

print(f'- Training performance: {np.mean(scores_train):.3f} ({np.std(scores_train, ddof=1):.3f})')
print(f'- Validation performance: {np.mean(scores_test):.3f} ({np.std(scores_test, ddof=1):.3f})')

- Training performance: 0.228 (0.015)
- Validation performance: 0.238 (0.024)


그럼, Cross-Validation에서 생성된 Split의 분포를 위에서 구현한 함수로 확인해보자.


In [ ]:
chart, stat = vis_and_stats(splitter, X, y)
display(chart)
display(stat)

alt.Chart(...)

,split,N.train,N.test,N.train.0,N.train.1,N.test.0,N.test.1,ratio_train,ratio_test
0,1-th split,600,400,291,309,199,201,0.941748,0.990050
1,2-th split,600,400,293,307,197,203,0.954397,0.970443
2,3-th split,600,400,286,314,204,196,0.910828,1.040816
3,4-th split,600,400,286,314,204,196,0.910828,1.040816
4,5-th split,600,400,291,309,199,201,0.941748,0.990050


보다시피, Startification이 적용되지 않은 일반적인 Hold-out의 경우 훈련 데이터셋과 검증 데이터셋의 레이블 분포가 다르다. 물론, 전체 데이터셋의 레이블 분포가 균형잡혀 있다면 이런 일이 크게 문제가 되지 않는다. 하지만, 레이블 분포가 굉장히 불균형하다면 어떨까?

In [ ]:
import numpy as np
from sklearn.datasets import make_classification
from sklearn.model_selection import ShuffleSplit
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import log_loss


X, y = make_classification(
    n_samples=1000, # 총 샘플 개수
    n_features=15, # 특성값 수
    n_redundant=1, # 다중공선성을 가지는 특성값 수
    n_clusters_per_class=1, # 레이블 값 별 데이터 군집의 수
    weights=[0.99, 0.01], # 레이블 값 비율
    flip_y=0, # 레이블 값에 적용할 노이즈 비율
    random_state=42
)

splitter = ShuffleSplit(
    n_splits=5, # 생성할 Split의 갯수
    test_size=0.4, # 전체 샘플의 40%를 검증 데이터로 활용
    random_state=42
)

scores_train, scores_test = [], []

# split 호출 시 훈련 데이터셋과 검증 데이터셋이 될 샘플의 인덱스들을 tuple 형식으로 반환하는 Generator가 생성된다.
for I_train, I_test in splitter.split(X, y):
    # 훈련 및 검증 데이터셋은 다음과 같은 방식으로 나누면 된다.
    X_train, X_test, y_train, y_test = X[I_train, :], X[I_test], y[I_train], y[I_test]

    # 훈련 데이터셋으로 모델을 훈련시킨다.
    model = LogisticRegression().fit(X=X_train, y=y_train)

    # 훈련이 잘 되었는지 확인해본다.
    score_train = log_loss(y_true=y_train, y_pred=model.predict_proba(X_train)[:, 1])

    # 검증 데이터셋을 통해 일반화 성능을 추정한다.
    score_test = log_loss(y_true=y_test, y_pred=model.predict_proba(X_test)[:, 1])

    scores_train.append(score_train)
    scores_test.append(score_test)

print(f'- Training performance: {np.mean(scores_train):.3f} ({np.std(scores_train, ddof=1):.3f})')
print(f'- Validation performance: {np.mean(scores_test):.3f} ({np.std(scores_test, ddof=1):.3f})')

- Training performance: 0.006 (0.001)
- Validation performance: 0.039 (0.019)


보다시피, 훈련 데이터셋과 검증 데이터셋 간의 성능이 크게 차이가 나는 것을 알 수 있다. 즉, 여기서 훈련한 모델은 훈련 데이터셋에 굉장히 과적합 되었으며, 우리가 알지못하는 테스트 데이터에 우리 모델을 적용했을 때 굉장히 나쁜 성능을 낼 것이라고 결론을 내릴 수 있다. 과연 이 결론이 옳을까?

Staratification을 적용해보자.

In [ ]:
import numpy as np
from sklearn.datasets import make_classification
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import log_loss


X, y = make_classification(
    n_samples=1000, # 총 샘플 개수
    n_features=15, # 특성값 수
    n_redundant=1, # 다중공선성을 가지는 특성값 수
    n_clusters_per_class=1, # 레이블 값 별 데이터 군집의 수
    weights=[0.99, 0.01], # 레이블 값 비율
    flip_y=0, # 레이블 값에 적용할 노이즈 비율
    random_state=42
)

splitter = StratifiedShuffleSplit(
    n_splits=5, # 생성할 Split의 갯수
    test_size=0.4, # 전체 샘플의 40%를 검증 데이터로 활용
    random_state=42
)

scores_train, scores_test = [], []

# split 호출 시 훈련 데이터셋과 검증 데이터셋이 될 샘플의 인덱스들을 tuple 형식으로 반환하는 Generator가 생성된다.
for I_train, I_test in splitter.split(X, y):
    # 훈련 및 검증 데이터셋은 다음과 같은 방식으로 나누면 된다.
    X_train, X_test, y_train, y_test = X[I_train, :], X[I_test], y[I_train], y[I_test]

    # 훈련 데이터셋으로 모델을 훈련시킨다.
    model = LogisticRegression().fit(X=X_train, y=y_train)

    # 훈련이 잘 되었는지 확인해본다.
    score_train = log_loss(y_true=y_train, y_pred=model.predict_proba(X_train)[:, 1])

    # 검증 데이터셋을 통해 일반화 성능을 추정한다.
    score_test = log_loss(y_true=y_test, y_pred=model.predict_proba(X_test)[:, 1])

    scores_train.append(score_train)
    scores_test.append(score_test)

print(f'- Training performance: {np.mean(scores_train):.3f} ({np.std(scores_train, ddof=1):.3f})')
print(f'- Validation performance: {np.mean(scores_test):.3f} ({np.std(scores_test, ddof=1):.3f})')

- Training performance: 0.011 (0.002)
- Validation performance: 0.015 (0.011)


In [ ]:
chart, stat = vis_and_stats(splitter, X, y)
display(chart)
display(stat)

alt.Chart(...)

,split,N.train,N.test,N.train.0,N.train.1,N.test.0,N.test.1,ratio_train,ratio_test
0,1-th split,600,400,594,6,396,4,99.0,99.0
1,2-th split,600,400,594,6,396,4,99.0,99.0
2,3-th split,600,400,594,6,396,4,99.0,99.0
3,4-th split,600,400,594,6,396,4,99.0,99.0
4,5-th split,600,400,594,6,396,4,99.0,99.0


보다시피, Stratification을 적용하게 되면 훈련 데이터셋과 검증 데이터셋의 레이블 분포가 동일하게 유지된다. 또한, 성능을 보았을 때도 상대적으로 훈련 데이터셋에 과적합되긴 했지만, Stratification을 적용하기 전과 비교했을 때 극적으로 차이가 나지 않는 것을 알 수 있다.

과연 어떠한 것을 선택해야할까? 역시나, 기계 학습 모델을 적용하는 문제 분야에 따라 다르다. 수집한 데이터셋(즉, 표본)에서의 레이블 분포가 모집단의 레이블 분포와 거의 일치할 것이라고 가정한다면, Stratification을 적용하는 것이 맞다. 하지만, 크게 다를것이라고 예상된다면 당연히 Stratification을 적용하지 않아야 한다.

하지만, 보통은 통계에서는 샘플의 평균과 모집단의 평균이 같을 것임을 가정한다. 마찬가지로, 레이블 분포 또한 샘플에서의 레이블 분포와 모집단의 레이블 분포가 같을 것임을 가정한다. 따라서, 보통은 Stratfication을 적용하는 편이다.

아래의 실습에서부터는 모두 Stratification을 적용하겠다.


## *k*-Fold CV
*k*-Fold Cross-validation은 각각 Stratification이 적용되지 않은 [sklearn.model_selection.KFold](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.KFold.html#sklearn.model_selection.KFold)과 Stratification이 적용된 [sklearn.model_selection.StratifiedKFold](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.StratifiedKFold.html#sklearn.model_selection.StratifiedKFold)을 사용하면 된다.

In [ ]:
import numpy as np
from sklearn.datasets import make_classification
from sklearn.model_selection import StratifiedKFold


X, y = make_classification(
    n_samples=1000, # 총 샘플 개수
    n_features=15, # 특성값 수
    n_redundant=1, # 다중공선성을 가지는 특성값 수
    n_clusters_per_class=1, # 레이블 값 별 데이터 군집의 수
    weights=[0.99, 0.01], # 레이블 값 비율
    flip_y=0, # 레이블 값에 적용할 노이즈 비율
    random_state=42
)

splitter = StratifiedKFold(
    n_splits=5, # 생성할 Split의 갯수 (또는 k)
    shuffle=False, # Split 생성 전 전체 데이터를 무작위로 섞을건지 여부
)

chart, stat = vis_and_stats(splitter, X, y)
display(chart)
display(stat)

alt.Chart(...)

,split,N.train,N.test,N.train.0,N.train.1,N.test.0,N.test.1,ratio_train,ratio_test
0,1-th split,800,200,792,8,198,2,99.0,99.0
1,2-th split,800,200,792,8,198,2,99.0,99.0
2,3-th split,800,200,792,8,198,2,99.0,99.0
3,4-th split,800,200,792,8,198,2,99.0,99.0
4,5-th split,800,200,792,8,198,2,99.0,99.0


시각화를 위해서 **shuffle**을 **False**로 했지만, **True**로 해서 전체 데이터를 섞어주는게 일반적이다.

In [ ]:
import numpy as np
from sklearn.datasets import make_classification
from sklearn.model_selection import StratifiedKFold


X, y = make_classification(
    n_samples=1000, # 총 샘플 개수
    n_features=15, # 특성값 수
    n_redundant=1, # 다중공선성을 가지는 특성값 수
    n_clusters_per_class=1, # 레이블 값 별 데이터 군집의 수
    weights=[0.99, 0.01], # 레이블 값 비율
    flip_y=0, # 레이블 값에 적용할 노이즈 비율
    random_state=42
)

splitter = StratifiedKFold(
    n_splits=5, # 생성할 Split의 갯수 (또는 k)
    shuffle=True, # Split 생성 전 전체 데이터를 무작위로 섞을건지 여부
    random_state=42
)

chart, stat = vis_and_stats(splitter, X, y)
display(chart)
display(stat)

alt.Chart(...)

,split,N.train,N.test,N.train.0,N.train.1,N.test.0,N.test.1,ratio_train,ratio_test
0,1-th split,800,200,792,8,198,2,99.0,99.0
1,2-th split,800,200,792,8,198,2,99.0,99.0
2,3-th split,800,200,792,8,198,2,99.0,99.0
3,4-th split,800,200,792,8,198,2,99.0,99.0
4,5-th split,800,200,792,8,198,2,99.0,99.0


## Repeated *k*-Fold CV
*k*-Fold를 여러번 반복하는 Repeated *k*-Fold는 매번 *k*개의 Split들을 생성하기 전에 전체 데이터를 무작위로 섞어준다. 당연히, 한번 *k*-Fold를 하는 것보다 더 나은 성능 추정이 가능하다.
이전 CV 방법과 마찬가지로, Stratification 적용 여부에 따라 [sklearn.model_selection.RepeatedKFold](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.RepeatedKFold.html#sklearn.model_selection.RepeatedKFold)과 [sklearn.model_selection.RepeatedStratifiedKFold](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.RepeatedStratifiedKFold.html#sklearn.model_selection.RepeatedStratifiedKFold) 의 두 가지 구현체가 존재한다.

In [ ]:
import numpy as np
from sklearn.datasets import make_classification
from sklearn.model_selection import RepeatedStratifiedKFold


X, y = make_classification(
    n_samples=1000, # 총 샘플 개수
    n_features=15, # 특성값 수
    n_redundant=1, # 다중공선성을 가지는 특성값 수
    n_clusters_per_class=1, # 레이블 값 별 데이터 군집의 수
    weights=[0.99, 0.01], # 레이블 값 비율
    flip_y=0, # 레이블 값에 적용할 노이즈 비율
    random_state=42
)

splitter = RepeatedStratifiedKFold(
    n_splits=5, # 생성할 Split의 갯수 (또는 k)
    n_repeats=2, # 반복 횟수; 즉 k-Fold들 두 번 반복하되, 매 반복마다 전체 데이터를 섞는다.
    random_state=42
)

chart, stat = vis_and_stats(splitter, X, y)
display(chart)
display(stat)

alt.Chart(...)

,split,N.train,N.test,N.train.0,N.train.1,N.test.0,N.test.1,ratio_train,ratio_test
0,1-th split,800,200,792,8,198,2,99.0,99.0
1,2-th split,800,200,792,8,198,2,99.0,99.0
2,3-th split,800,200,792,8,198,2,99.0,99.0
3,4-th split,800,200,792,8,198,2,99.0,99.0
4,5-th split,800,200,792,8,198,2,99.0,99.0
5,6-th split,800,200,792,8,198,2,99.0,99.0
6,7-th split,800,200,792,8,198,2,99.0,99.0
7,8-th split,800,200,792,8,198,2,99.0,99.0
8,9-th split,800,200,792,8,198,2,99.0,99.0
9,10-th split,800,200,792,8,198,2,99.0,99.0


## Leave-One-Out CV
샘플 하나를 검증 데이터셋으로 삼고, 나머지 모두를 훈련 데이터셋으로 삼는 Leave-One-Out은 [sklearn.model_selection.LeaveOneOut](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.LeaveOneOut.html#sklearn.model_selection.LeaveOneOut)에 구현되어 있다. 샘플 9개만 생성해서 해보자.

In [ ]:
import numpy as np
from sklearn.datasets import make_classification
from sklearn.model_selection import LeaveOneOut


X, y = make_classification(
    n_samples=9, # 총 샘플 개수
    n_features=2, # 특성값 수
    n_redundant=0, # 다중공선성을 가지는 특성값 수
    n_clusters_per_class=1, # 레이블 값 별 데이터 군집의 수
    weights=[0.5, 0.5], # 레이블 값 비율
    flip_y=0, # 레이블 값에 적용할 노이즈 비율
    random_state=42
)

splitter = LeaveOneOut()
chart, stat = vis_and_stats(splitter, X, y, tick_options=dict(thickness=15))
display(chart)
display(stat)

alt.Chart(...)

,split,N.train,N.test,N.train.0,N.train.1,N.test.0,N.test.1,ratio_train,ratio_test
0,1-th split,8,1,4,4,1.0,NaN,1.000000,NaN
1,2-th split,8,1,5,3,NaN,1.0,1.666667,NaN
2,3-th split,8,1,4,4,1.0,NaN,1.000000,NaN
3,4-th split,8,1,4,4,1.0,NaN,1.000000,NaN
4,5-th split,8,1,4,4,1.0,NaN,1.000000,NaN
5,6-th split,8,1,5,3,NaN,1.0,1.666667,NaN
6,7-th split,8,1,4,4,1.0,NaN,1.000000,NaN
7,8-th split,8,1,5,3,NaN,1.0,1.666667,NaN
8,9-th split,8,1,5,3,NaN,1.0,1.666667,NaN


## Leave-*P*-Out

Leave-Out-Out CV의 보다 일반적인 방법인 [sklearn.model_selection.LeavePOut](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.LeavePOut.html#sklearn.model_selection.LeavePOut)도 해보자.

In [ ]:
import numpy as np
from sklearn.datasets import make_classification
from sklearn.model_selection import LeavePOut


X, y = make_classification(
    n_samples=5, # 총 샘플 개수
    n_features=2, # 특성값 수
    n_redundant=0, # 다중공선성을 가지는 특성값 수
    n_clusters_per_class=1, # 레이블 값 별 데이터 군집의 수
    weights=[0.5, 0.5], # 레이블 값 비율
    flip_y=0, # 레이블 값에 적용할 노이즈 비율
    random_state=42
)

splitter = LeavePOut(
    p=3
)
chart, stat = vis_and_stats(splitter, X, y, tick_options=dict(thickness=15))
display(chart)
display(stat)

alt.Chart(...)

,split,N.train,N.test,N.train.0,N.test.0,N.test.1,N.train.1,ratio_train,ratio_test
0,1-th split,2,3,2.0,1,2.0,NaN,NaN,0.5
1,2-th split,2,3,1.0,2,1.0,1.0,1.0,2.0
2,3-th split,2,3,1.0,2,1.0,1.0,1.0,2.0
3,4-th split,2,3,1.0,2,1.0,1.0,1.0,2.0
4,5-th split,2,3,1.0,2,1.0,1.0,1.0,2.0
5,6-th split,2,3,NaN,3,NaN,2.0,NaN,NaN
6,7-th split,2,3,2.0,1,2.0,NaN,NaN,0.5
7,8-th split,2,3,2.0,1,2.0,NaN,NaN,0.5
8,9-th split,2,3,1.0,2,1.0,1.0,1.0,2.0
9,10-th split,2,3,1.0,2,1.0,1.0,1.0,2.0


총 10개의 Split이 생성되었는데, 그 이유는 5개의 샘플 중 3개를 순서 없이 뽑는 방법(조합)을 모두 찾는 것과 같기 때문이다. 즉, $\frac{5 * 4 * 3}{3 * 2 * 1} = \frac{60}{6}=10 $과 같다.

## Leave-One-Group-Out
이번에는 그룹별로 Split을 나눠보자. 먼저, 한 그룹을 검증 데이터셋, 남은 그룹들을 훈련 데이터셋으로 활용하는 Leave-One-Group-Out이다. [sklearn.model_selection.LeaveOneGroupOut](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.LeaveOneGroupOut.html#sklearn.model_selection.LeaveOneGroupOut)에 구현되어 있다.

In [ ]:
import numpy as np
from sklearn.datasets import make_classification
from sklearn.model_selection import LeaveOneGroupOut


X, y = make_classification(
    n_samples=30, # 총 샘플 개수
    n_features=2, # 특성값 수
    n_redundant=0, # 다중공선성을 가지는 특성값 수
    n_clusters_per_class=1, # 레이블 값 별 데이터 군집의 수
    weights=[0.5, 0.5], # 레이블 값 비율
    flip_y=0, # 레이블 값에 적용할 노이즈 비율
    random_state=42
)

# 가상으로 아래처럼 그룹을 만들어보자
group = np.asarray(['A'] * 5 + ['B'] * 3 + ['C'] * 8 + ['D'] * 4 + ['E'] * 7 + ['F'] * 3)

splitter = LeaveOneGroupOut()
chart, stat = vis_and_stats(splitter, X, y, group=group, tick_options=dict(thickness=10))
display(chart)
display(stat)

alt.VConcatChart(...)

,split,N.train,N.test,N.train.0,N.train.1,N.test.0,N.test.1,ratio_train,ratio_test
0,1-th split,25,5,12,13,3,2,0.923077,1.50
1,2-th split,27,3,14,13,1,2,1.076923,0.50
2,3-th split,22,8,12,10,3,5,1.200000,0.60
3,4-th split,26,4,12,14,3,1,0.857143,3.00
4,5-th split,23,7,12,11,3,4,1.090909,0.75
5,6-th split,27,3,13,14,2,1,0.928571,2.00


## Leave-*P*-Group-Out CV
Leave-*P*-Out과 유사하게, 전체 그룹 중에서 P개의 그룹을 순서 없이 뽑는  Leave-*P*-Group-Out도 당연히 사용할 수 있. [sklearn.model_selection.LeavePGroupsOut](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.LeavePGroupsOut.html#sklearn.model_selection.LeavePGroupsOut)을 확인해보자.

In [ ]:
import numpy as np
from sklearn.datasets import make_classification
from sklearn.model_selection import LeavePGroupsOut


X, y = make_classification(
    n_samples=30, # 총 샘플 개수
    n_features=2, # 특성값 수
    n_redundant=0, # 다중공선성을 가지는 특성값 수
    n_clusters_per_class=1, # 레이블 값 별 데이터 군집의 수
    weights=[0.5, 0.5], # 레이블 값 비율
    flip_y=0, # 레이블 값에 적용할 노이즈 비율
    random_state=42
)

# 가상으로 아래처럼 그룹을 만들어보자
group = np.asarray(['A'] * 5 + ['B'] * 3 + ['C'] * 8 + ['D'] * 4 + ['E'] * 7 + ['F'] * 3)

splitter = LeavePGroupsOut(
    n_groups=2
)
chart, stat = vis_and_stats(splitter, X, y, group=group, tick_options=dict(thickness=10))
display(chart)
display(stat)

alt.VConcatChart(...)

,split,N.train,N.test,N.train.0,N.train.1,N.test.0,N.test.1,ratio_train,ratio_test
0,1-th split,22,8,11,11,4,4,1.000000,1.000000
1,2-th split,17,13,9,8,6,7,1.125000,0.857143
2,3-th split,21,9,9,12,6,3,0.750000,2.000000
3,4-th split,18,12,9,9,6,6,1.000000,1.000000
4,5-th split,22,8,10,12,5,3,0.833333,1.666667
5,6-th split,19,11,11,8,4,7,1.375000,0.571429
6,7-th split,23,7,11,12,4,3,0.916667,1.333333
7,8-th split,20,10,11,9,4,6,1.222222,0.666667
8,9-th split,24,6,12,12,3,3,1.000000,1.000000
9,10-th split,18,12,9,9,6,6,1.000000,1.000000


## Group *k*-Fold CV
이번엔 Group *k*-Fold를 사용해 그룹을 다른 방식으로 묶어보자. 일반 *k*-Fold와 마찬가지로, scikit-learn에서는 Stratification을 적용하지 않은 [sklearn.model_selection.GroupKFold](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GroupKFold.html#sklearn.model_selection.GroupKFold)과 Stratification을 적용한 [sklearn.model_selection.StratifiedGroupKFold](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.StratifiedGroupKFold.html#sklearn.model_selection.StratifiedGroupKFold)이 각각 구현되어 있다.

In [ ]:
import numpy as np
from sklearn.datasets import make_classification
from sklearn.model_selection import StratifiedGroupKFold


X, y = make_classification(
    n_samples=30, # 총 샘플 개수
    n_features=2, # 특성값 수
    n_redundant=0, # 다중공선성을 가지는 특성값 수
    n_clusters_per_class=1, # 레이블 값 별 데이터 군집의 수
    weights=[0.3, 0.7], # 레이블 값 비율
    flip_y=0, # 레이블 값에 적용할 노이즈 비율
    random_state=42
)

# 가상으로 아래처럼 그룹을 만들어보자
group = np.asarray(['A'] * 5 + ['B'] * 3 + ['C'] * 8 + ['D'] * 4 + ['E'] * 7 + ['F'] * 3)

splitter = StratifiedGroupKFold(
    n_splits=4,
    shuffle=True,
    random_state=42
)
chart, stat = vis_and_stats(splitter, X, y, group=group, tick_options=dict(thickness=10))
display(chart)
display(stat)

alt.VConcatChart(...)

,split,N.train,N.test,N.train.0,N.train.1,N.test.0,N.test.1,ratio_train,ratio_test
0,1-th split,18,12,6,12,3,9,0.500000,0.333333
1,2-th split,22,8,7,15,2,6,0.466667,0.333333
2,3-th split,23,7,7,16,2,5,0.437500,0.400000
3,4-th split,27,3,7,20,2,1,0.350000,2.000000


위에서 볼 수 있듯이, Stratification을 적용했더라도 훈련 데이터셋과 검증 데이터셋 간의 레이블 분포가 차이나는 것을 알 수 있다. 그룹 단위로 묶는 Group *k*-Fold의 특성상,  묶인 그룹들의 레이블 분포가 불균형하면 이런 일이 발생할 수 밖에 없다. 어디까지나 최대한 레이블 분포가 덜 차이나는 훈련 데이터셋/검증 데이터셋의 짝을 찾을 뿐이다.

## Time-Series CV
이번엔 시간에 따라 훈련 데이터셋과 검증 데이터셋을 구분하는 Time-Series CV를 적용해보자. [sklearn.model_selection.TimeSeriesSplit](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.TimeSeriesSplit.html#sklearn.model_selection.TimeSeriesSplit)을 사용하면 된다.

In [ ]:
import numpy as np
from sklearn.datasets import make_classification
from sklearn.model_selection import TimeSeriesSplit


X, y = make_classification(
    n_samples=30, # 총 샘플 개수
    n_features=2, # 특성값 수
    n_redundant=0, # 다중공선성을 가지는 특성값 수
    n_clusters_per_class=1, # 레이블 값 별 데이터 군집의 수
    weights=[0.3, 0.7], # 레이블 값 비율
    flip_y=0, # 레이블 값에 적용할 노이즈 비율
    random_state=42
)

splitter = TimeSeriesSplit(
    test_size=5 # 하나의 시간 단위에 샘플이 5개가 존재한다고 가정하자.
)
chart, stat = vis_and_stats(splitter, X, y, tick_options=dict(thickness=10))
display(chart)
display(stat)

alt.Chart(...)

,split,N.train,N.test,N.train.0,N.train.1,N.test.0,N.test.1,ratio_train,ratio_test
0,1-th split,5,5,1,4,1.0,4,0.250000,0.250000
1,2-th split,10,5,2,8,NaN,5,0.250000,NaN
2,3-th split,15,5,2,13,3.0,2,0.153846,1.500000
3,4-th split,20,5,5,15,2.0,3,0.333333,0.666667
4,5-th split,25,5,7,18,2.0,3,0.388889,0.666667
